In [2]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1208484/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [4]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [6]:
analize = Analizer(0.3)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_8_30,0.377845,-0.096074,-0.161832,-4.874351e+18,-0.366285,0.075745,0.650677,0.206519,0.036465,0.121492,0.099268,0.275219,-0.296156,0.277429,135.160759,319.519083,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_8_29,0.376516,-0.096047,-0.159421,-4.837110e+18,-0.362309,0.075907,0.650661,0.206090,0.036187,0.121138,0.099569,0.275512,-0.298925,0.277725,135.156491,319.514815,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_9_21,0.376129,0.027538,-1.726576,-3.360134e-01,-0.538602,0.075954,0.577295,0.067007,0.192524,0.129765,0.094611,0.275598,-0.299732,0.277812,135.155249,319.513573,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_8_31,0.375225,-0.097612,-0.189249,-4.836901e+18,-0.392113,0.076064,0.651590,0.211392,0.036185,0.123789,0.099122,0.275798,-0.301616,0.278013,135.152352,319.510676,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_9_20,0.374516,0.027951,-1.776504,-3.318195e-01,-0.542293,0.076151,0.577050,0.068234,0.191919,0.130077,0.095179,0.275954,-0.303092,0.278171,135.150085,319.508409,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,model_1_4_45,-0.288786,-0.150917,-5.046490,-1.853108e+00,-3.030610,0.156905,0.683234,0.568725,0.356893,0.462809,0.106384,0.396113,-1.684971,0.399295,133.704224,318.062548,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
146,model_1_4_46,-0.316521,-0.152548,-5.198699,-1.929189e+00,-3.134393,0.160282,0.684202,0.583041,0.366410,0.474726,0.107111,0.400352,-1.742751,0.403568,133.661641,318.019965,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
147,model_1_4_47,-0.385219,-0.145153,-5.552968,-2.137419e+00,-3.392918,0.168646,0.679812,0.616363,0.392457,0.504410,0.108261,0.410665,-1.885873,0.413964,133.559910,317.918234,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
148,model_1_4_48,-0.424783,-0.171962,-5.619786,-2.364387e+00,-3.543915,0.173463,0.695727,0.622648,0.420849,0.521748,0.109090,0.416488,-1.968298,0.419834,133.503587,317.861911,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
